**(1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.**

За основу корпуса я беру стихотворения Велимира Хлебникова. Их особенностью является большое количество придуманных им самим слов. Казалось бы, это не должно препятствовать нашей задаче, т.к. морфемы в его стихотворениях как-раз помогают складывать общий смысл, однако погрешность разметки все равно существенная.

In [127]:
with open('corpus.txt', 'r', encoding='utf-8') as fr, open('words.csv', 'w', encoding='utf-8') as fw:
    for line in fr:
        for word in line.split():
            fw.write(word)
            fw.write('\n')

Я разметила корпус тегами НКРЯ, так как они кажутся мне удобными, соответственно, теги других парсеров буду выравнивать под формат НКРЯ

**(3 балла) Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.**

*STANZA POS-tagging*

In [15]:
!pip3 install stanza

In [128]:
import stanza
stanza.download('ru')

2022-10-11 22:30:14 INFO: Downloading default packages for language: ru (Russian)...
2022-10-11 22:30:17 INFO: File exists: C:\Users\varva\stanza_resources\ru\default.zip
2022-10-11 22:30:22 INFO: Finished downloading models and saved to C:\Users\varva\stanza_resources.


In [130]:
ppln = stanza.Pipeline('ru', processors='tokenize,pos')
with open('corpus.txt', 'r', encoding='utf-8') as fr_stanza, open('words_stanza.csv', 'w', encoding='utf-8') as fw_stanza:
    doc = ppln(fr_stanza.read())
    for snt in doc.sentences:
        for word in snt.words:
            fw_stanza.write(word.text)
            fw_stanza.write(',')
            fw_stanza.write(word.upos)
            fw_stanza.write('\n')

2022-10-11 22:30:44 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2022-10-11 22:30:44 INFO: Use device: cpu
2022-10-11 22:30:44 INFO: Loading: tokenize
2022-10-11 22:30:44 INFO: Loading: pos
2022-10-11 22:30:45 INFO: Done loading processors!


*PYMORPHY POS-tagging*

In [4]:
!pip install pymorphy2[fast]

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'dawg' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for DAWG
  error: subprocess-exited-with-error
  
  Running setup.py install for DAWG did not run successfully.
  exit code: 1
  
  [5 lines of output]
  running install
  running build
  running build_ext
  building 'dawg' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely no

  Using cached DAWG-0.8.0.tar.gz (371 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for DAWG
Failed to build DAWG
  Running setup.py install for DAWG: started
  Running setup.py install for DAWG: finished with status 'error'


In [136]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [137]:
import nltk
from nltk import word_tokenize

In [138]:
with open('words.csv', 'r', encoding='utf-8') as fr_pymorphy, open('words_pymorphy.csv', 'w', encoding='utf-8') as fw_pymorphy:
    tokens = word_tokenize(fr_pymorphy.read())
    for token in tokens:
        fw_pymorphy.write(token)
        fw_pymorphy.write(' ')
        analysis = morph.parse(token)[0]
        tag = analysis.tag
        fw_pymorphy.write(str(tag))
        fw_pymorphy.write('\n')

*NATASHA POS-tagging*

In [51]:
!pip install natasha

     ---------------------------------------- 34.4/34.4 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 41.1/41.1 KB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 49.4/49.4 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26101 sha256=88a34abbf8f9a74314f3976272097c718a7dc901d7b54970b5f071220a886f9b
  Stored in directory: c:\users\varva\appdata\local\pip\cache\wheels\45\23\de\5789a92962483fd33cb06674792b9697c1b3766d7c7742830e
Successfully built intervaltree


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [139]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [145]:
with open('corpus.txt', 'r', encoding='utf-8') as fr:
    text = fr.read()
    doc = Doc(text)

In [142]:
doc.segment(segmenter)
display(doc.tokens[:5])

[DocToken(stop=4, text='Росу'),
 DocToken(start=5, stop=13, text='вишневую'),
 DocToken(start=14, stop=18, text='меча'),
 DocToken(start=19, stop=21, text='Ты'),
 DocToken(start=22, stop=28, text='сушишь')]

In [143]:
doc.segment(segmenter)
display(doc.sents[:5])

[DocSent(stop=47, text='Росу вишневую меча\nТы сушишь волосом волнистым.', tokens=[...]),
 DocSent(start=48, stop=103, text='А здесь из смеха палача\nПриходит тот, чей смех н..., tokens=[...]),
 DocSent(start=104, stop=203, text='То черноглазою гадалкой,\nМногоглагольная, молчиш..., tokens=[...]),
 DocSent(start=204, stop=253, text='Он умер, подымая бивни,\nОпять на небе виден Хорс..., tokens=[...]),
 DocSent(start=254, stop=306, text='Его живого знали ливни —\nТеперь он глыба, он зам..., tokens=[...])]

In [144]:
doc.tag_morph(morph_tagger)
display(doc.tokens)

[DocToken(stop=4, text='Росу', pos='ADJ', feats=<Gen,Pos,Masc,Sing>),
 DocToken(start=5, stop=13, text='вишневую', pos='ADJ', feats=<Gen,Pos,Masc,Sing>),
 DocToken(start=14, stop=18, text='меча', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=19, stop=21, text='Ты', pos='PRON', feats=<Nom,Sing,2>),
 DocToken(start=22, stop=28, text='сушишь', pos='VERB', feats=<Perf,Masc,Ind,Sing,Past,Fin,Act>),
 DocToken(start=29, stop=36, text='волосом', pos='ADJ', feats=<Acc,Pos,Fem,Sing>),
 DocToken(start=37, stop=46, text='волнистым', pos='NOUN', feats=<Inan,Loc,Masc,Sing>),
 DocToken(start=46, stop=47, text='.', pos='PUNCT'),
 DocToken(start=48, stop=49, text='А', pos='CCONJ'),
 DocToken(start=50, stop=55, text='здесь', pos='ADV', feats=<Pos>),
 DocToken(start=56, stop=58, text='из', pos='ADP'),
 DocToken(start=59, stop=64, text='смеха', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=65, stop=71, text='палача', pos='NOUN', feats=<Anim,Gen,Masc,Sing>),
 DocToken(start=72, stop=8

**(2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.**

Составляю таблицы соответствий тегов stanza, pymorphy и natasha с тегами НКРЯ; нужно перевести все в один формат, чтобы иметь возможность измерить accuracy

In [147]:
pos_from_stanza_to_gr_table = {
            "ADJ": "A",
            "ADP": "PR",
            "ADV": "ADV",
            "AUX": "AUX",
            "CCONJ": "CONJ",
            "DET": "DET",
            "INTJ": "INTJ",
            "NOUN": "S",
            "NUM": "NUM",
            "PART": "PART",
            "PRON": "PRO",
            "PROPN": "persn",
            "SCONJ": "CONJ",
            "VERB": "V",
            "X": "X",
            "PUNCT": "PUNCT"
        }

In [148]:
pos_from_pymorphy_to_gr_table = {
            "ADJ": "A",
            "ADP": "PR",
            "ADV": "ADV",
            "AUX": "AUX",
            "CCONJ": "CONJ",
            "DET": "DET",
            "INTJ": "INTJ",
            "NOUN": "S",
            "NUM": "NUM",
            "PART": "PART",
            "PRON": "PRO",
            "PROPN": "persn",
            "SCONJ": "CONJ",
            "VERB": "V",
            "X": "X",
            "PUNCT": "PUNCT"
        }

In [149]:
pos_from_natasha_to_gr_table = {
            "ADJ": "A",
            "ADP": "PR",
            "ADV": "ADV",
            "AUX": "AUX",
            "CCONJ": "CONJ",
            "DET": "DET",
            "INTJ": "INTJ",
            "NOUN": "S",
            "NUM": "NUM",
            "PART": "PART",
            "PRON": "PRO",
            "PROPN": "persn",
            "SCONJ": "CONJ",
            "VERB": "V",
            "X": "X",
            "PUNCT": "PUNCT"
        }

In [150]:
# потом нужно сделать функцию для перевода тегов в один формат

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# потом делаем что-то такое и получаем ответ
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(y_true, y_pred)